# Deep COVID: An Exposition

Throughout our report, we present the contents of 'Deep-COVID: Predicting COVID-19 from chest X-ray images using deep transfer learning' (Deep-COVID, for short) by Minaee et al. We start with an introduction to the research question and overview of the data. Following this, we fit a logistic regression to the classification task. This will be used as our baseline model to compare the models used in the paper against. It also serves as an introduction to the basic concepts of Artificial Neural Networks (ANNs). Using this as a segue, we, next, discuss the basic concepts involved in designing, and training ANN's. Once this is done, we introduce the concepts particular to Convolutional Neural Networks (CNNs). Then, we briefly discuss transfer learning and present the models used in the paper. Finally, we review their model evaluation metrics in comparison with eachother, our initial regression model, current biological-based tests, and discuss potential future work.

## The Goal of Deep-COVID and the Data
### The Goal

The COVID pandemic represents one of the biggest challenges currently facing the United States and the rest of the world. Before vaccination is sufficiently deployed and adopted, efforts towards treatment and prevention are of much importance. One of the most effective tools we have at our disposal is accurate and speedy detection. Diagnosing cases from radiography and radiology image is, perhaps, one of the fastest forms of detection (Minaee et al., 2020). In their paper, Minaee et al. apply transfer learning to fit famous ANNs, including ResNet18, ResNet50, SqueezeNet, and DenseNet-121, to identify COVID-19 disease in analyzed chest X-ray images. Results were promising and it is the hope that these methods can be used to more accurately and quickly identify cases in exposed individuals. We'll discuss, in more detail, the concepts of transfer learning, as well as the basic concepts necessary to understand how the ANNs used are structured. First, however, let's become more acquainted with the data used for training and testing.

### The Data

The authors of Deep-COVID were nice enough to share their data along with a description of how it was collected, labeled, and organized in a Github repository (https://github.com/shervinmin/DeepCovid/tree/master/data).  Still, we will give a brief synopsis in this section.

The data used by the authors comes from two sources: 
* "Covid-Chestxray-Dataset", for COVID-positive samples
* "ChexPert Dataset", for COVID-negative samples.

<img src = "Sample_images_visualization.jpg" alt = "Visualizations of Data" style = "width : 300px;"/>

From the Covid-Chestxray-Dataset, only images that were selected to have a clear sign of COVID by the authors' board-certified radiologists were used: a total of 71 images, split 31 for training and 40 for testing. Several data-augmentation techniques (as well as over-sampling) were used to increase the number of COVID-19 samples, leading to final training/test split counts of:

<figure>
    <img src = "Data_train_test_counts_table.jpg" alt = "Data_train_test_counts_table" style = "width : 300px;"/>
    <figcaption style = "text-align : center;"> Data train/test split summary (Minaee et al., 2020) </figcaption>
</figure>

We note that one of the limitations of this project is the absence of a larger quantity of data, especially verified COVID-positive images. With the help of more data, we could hope for even better results. Now let's turn to our first model, a logistic regression model.

## Logistic Regression: A Baseline Model and Segue into ANNs

### Introduction and Defining the Model

In this section, we will fit a logitstic regression classifier to a subset of the author's data. This will serve two purposes. On the one hand, it will provide a baseline model to compare against the models used in Deep-COVID. And, on the other, it will serve as a segue into ANNs. Let us briefly recall the functional form of a logistic regression model. Namely, given a binary classification problem (say the two classes are 0 or 1) and a data point $\mathbf{x}$, the probability that $\mathbf{x}$ belongs to class 1 is given by:

$$P(y_i = 1 | \mathbf{x}) = \frac{1}{1 + e ^ {-\mathbf{w}^T \mathbf{x}}}.$$

This functional form can be thought of as the composition of two component functional forms. Namely, a linear component, $\mathbf{z}_i = \mathbf{w} ^ T \mathbf{x} = w_1x_1 + ... + w_px_p$ and a non-linear component, $a_i = \sigma(\mathbf{a}_i)$, where $y_i = a_i$ and $\sigma(x) = \frac{1}{1 + e ^ {-x}}$ denotes a sigmoid function, which we will refer to as a sigmoid *activation* function. Thus, graphically, we can think of our model as follows:

<figure>
    <img src = "Single_Perceptron.png" alt = "Single Perceptron" style = "width : 300px" />
    <figcaption style = "text-align : center;"> Perceptron architecture (Deshpande 2020) </figcaption>
</figure>

In order to train a model of this form to our classification task, we need to define how our input layer will look, i.e. what are our features. In the case of image classification tasks, it is common to use a matrix (2 dimensional for grey-scale images and 3 dimensional for color images) of pixel values of the images as the features. We can then take this matrix and "flatten" it, meaning that we convert it into a 1-Dimensional array (the form necessary for our intput layer above. Furthermore, we assign to each entry of the resulting vector, i.e. each pixel value a weight, and include it in the calculation of our linear component ("Sum" in the above graph). This means our model is "fully connected". Now that we've defined the functional form of our model and it's architecture, let's discuss how we will train it using our training data set. 

### Gradient Based Learning

Most machine learning algorithms, including deep learning algorithms, involve minimizing an objective function, $f(x)$, i.e. finding $x ^ * = \textrm{argmin} f(x)$. If $f(x)$ is differentiable, one common optimization method involves iteratively taking steps in the feature space such that, in each step, we move in a direction likely to furthest reduce $f(x)$. In other words, we want to find the direction, $\mathbf{u}$ such that the directional derivative $D_\mathbf{u} f(x)$ is minimized. (This is an intermediate optimization problem we solve to solve the primary optimization problem of minimizing $f$.) Then, we take a step of a specifized size, known as the "learning rate" (which we'll denote by $\epsilon$) in this direction and repeat.

In order to solve this intermediate optimization propblem, recall how partial derivatives are computed. Namely, given a direction $\mathbf{u}$, the directional derivative of $f(x)$ in the direction of $\mathbf{u}$ at a point, $\mathbf{x_0}$, is given by:

$$D_\mathbf{u} f(\mathbf{x_0}) = \mathbf{u} ^ T \nabla f(\mathbf{x_0})$$

where $\nabla f(\mathbf{x_0})$ is the gradient of $f$ at $\mathbf{x_0}$.

For the purposes of our intermediate optimization problem, we can assume (without loss of generality) that $u$ is a unit vector. Thus, $D_\mathbf{u} f(\mathbf{x_0})$, can be interpreted as the signed length of the projection of $\nabla f(\mathbf{x_0})$ onto the span of $\mathbf{u}$. Using this interpretation, we can now see that the solution to our intermediate problem is given by $-\nabla f(\mathbf{x_0})$. Using this, at each step in our optimization algorithm, we update our choice of $\mathbf{x}$ as follows:

$$\mathbf{x ^ \prime} = \mathbf{x} - \epsilon \nabla f(\mathbf{x}).$$

This process is known as "gradient descent", as we are descending down the objective function according to the gradient. While this constitutes the basis for many popular optimization algorithms used in deep learning, it is (by itself) often not the preferred method. In the next section, we will explain, briefly, some of the defects of the algorithm as well as popular alternatives or fixes.

### Improving Gradient Descent

Gradient descent uses the gradient of $f(x)$ to determine a step direction. In other words, it uses the first derivatives of $f(x)$, making it a "first-order" optimization method. This presents potential problems, including making choosing an appropriate learning rate very difficult, and an innability to effectively deal with saddle points. To illustrate the former, consider the one dimensional case, i.e. $x \in \mathbb{R}$ and $f(x) \in \mathbb{R}$. According to gradient descent, we move with a fixed learning rate in the direction of $f ^ {\prime}(x)$ which is smallest (i.e. negative). If, as we are approaching a local minima, $f ^ {\prime \prime} (x)$ is very large, we run the risk of over shooting the minimum with a larger learning rate, while smaller step sizes can cause the algorithm to be inefficient and take many steps to converge.

<figure>
    <img src = "one_dimensional_GD_step_sizes.png" alt = "first order step size issue" style = "width : 350px" />
    <figcaption style = "text-align : center;"> Illustration of poor step sizes in one dimensional gradient descent (Varma &amp; Das, 2018) </figcaption>
</figure>

This defect persists, analogously, in multiple dimensions. In the image below, we view the trajectory of gradient descent in multiple dimensions using a contour plot representation. Note that the curvature of the objective function is largest in the $[1 \textrm{, } 1] ^ T$ direction and smallest in the $[1 \textrm{, } -1] ^ T$ direction. The objectice function resembles a long canyon (Goodfellow, Bengio & Courville, 2017). Due to a step size which is too large, gradient descent overshoots the bottom of the canyon, having to then descend the opposite canyon wall in the next iteration.

<figure>
    <img src = "inefficient_GD_multivariate.jpg" alt = "multivariate step size defect" style = "width : 250px;"> 
    <figcaption style = "text-align : center;"> A view of gradient descent on an objective function with ill-conditioned Hessian Matrix (Goodfellow, Bengio, & Courville, 2017) </figcaption>
</figure>

One approach to addressing these defects is to use information from the second derivative, or Hessian Matrix (the multivariate equivalent) to modify our step direction, making our approach "second-order". Namely, Newton's Method accomlishes this and can sometimes lead to a critical point much faster than gradient descent. 

Another approach is to smooth out the trajectory of gradient descent by using exponentially weighted averages of gradients, $V_{dW}$:

\begin{align*}
    V_{dW} &= 0 &&\textrm{ as an initial value.} \\
    V_{dW} &= \beta V_{dW} + (1 - \beta)dW &&\textrm{ for every successive iteration.}
\end{align*}

And model parameters are updaed by $\theta = \theta - V_{dW}$. Here $\beta$ is a parameter of the algorithm to be set. Intuitively, we can think of this algorithm of taking an approximate exponentially weighted average of the previous $\frac{1}{1 - \beta}$ terms. While this formulation is intuitive to understand, often, in practice, the update expression is written as $V_{dW} = \alpha V_{dW} - \epsilon dW$ and then parameters are updated by $\theta = \theta + V_{dW}$ (where $\epsilon$ need not equal $(1 + \alpha)$). This approach is known as gradient descent with "momentum". 

Yet another approach, known as RMSprop, modifies the learning rate in each direction differently and inversely proportional to the square root of an exponentially weighted average of the appropriate directional derivative. Namely:

\begin{align*}
    V_{dW} &= \beta V_{dW} + (1 - \beta)dW ^ 2 \textrm{ where } dW \textrm{ is squared elementwise.}
\end{align*}

And model parameters are updated by:

\begin{align*}
    \theta = \theta - \frac{\epsilon}{\sqrt{V_{dW}}} dW \textrm{ where division and multiplication are performed element wise.}
\end{align*}

One final approach we discuss is known as ADAM optimization.

In the case of binary classification, it is common to use the binary cross-entropy loss function as $f(x)$. Namely, we want to minimize:

$$f(x) = -\frac{1}{N}\sum_{i = 1} ^ N y_i \cdot \textrm{log}(p(y_i)) + (1 - y_i) \cdot \textrm{log}(1 - p(y_i))$$

In [4]:
##### Import the necessary modules #####

from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import RMSprop
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import random
import numpy as np

## Load in the training and testing images using ImageDataGenerator
# Training images
train_datagen = ImageDataGenerator(width_shift_range = 0.1)
train_dir = 'C:/Users/Owner/Documents/STAT 6750/Project/data_upload_v2/train'

train_covid_dir = 'C:/Users/Owner/Documents/STAT 6750/Project/data_upload_v2/train/covid'
train_covid_names = os.listdir(train_covid_dir)

train_non_dir = 'C:/Users/Owner/Documents/STAT 6750/Project/data_upload_v2/train/non'
train_non_names = os.listdir(train_non_dir)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = (250, 250),
    batch_size = 20,
    class_mode = 'binary'
)

#Testing images
test_datagen = ImageDataGenerator(rescale = 1./255)
validation_dir = 'C:/Users/Owner/Documents/STAT 6750/Project/data_upload_v2/test'

validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size = (250, 250),
    batch_size = 20,
    class_mode = 'binary'
)

## Define a logistic regression model as a perceptron with sigmoid activation function
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape = (250, 250, 3)),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(1, activation = 'sigmoid')
])

opt = keras.optimizers.Adam(learning_rate=0.0001)

model.compile(optimizer = opt,
              loss = 'binary_crossentropy',
              metrics = ['binary_accuracy', 'TruePositives', 'TrueNegatives'])

model.summary()

Found 664 images belonging to 2 classes.
Found 3100 images belonging to 2 classes.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 187500)            0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 187500)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 187501    
Total params: 187,501
Trainable params: 187,501
Non-trainable params: 0
_________________________________________________________________


## Introduction to ANNs and Deep Learning

## Introduction to CNNs

## Models Utilized in Minaee Et al.

## Evaluation Metrics and Future Work

## Bibliography

1) Minaee, Shervin et al. “Deep-COVID: Predicting COVID-19 from chest X-ray images using deep transfer learning.” Medical image analysis vol. 65 (2020): 101794. doi:10.1016/j.media.2020.101794

2) Goodfellow, I., Bengio, Y., &amp; Courville, A. (2017). Deep learning. Cambridge, MA: MIT Press.

3) Varma, S., &amp; Das, S. (2018, September 27). Deep Learning. Retrieved November 24, 2020, from https://srdas.github.io/DLBook/

4) Deshpande, M. (2020, September 29). Perceptrons: The First Neural Networks. Retrieved November 23, 2020, from https://pythonmachinelearning.pro/perceptrons-the-first-neural-networks/